In [1]:
#Created by Leoanrdo G. Dettori on 11/14/2022
#This program takes in the chemical shifts values and suggests a selection of likely amino acids
#according to data from BMRB (Biological Magnetic Resonance Data Bank)

file_path_BMRB = "C:/Users/leogd/cs_stat_aa_full.csv"


# ========================================================================== #

print("Reading BMRB Data Sheet..."+'\n')

#Reads BMRB data sheet
#This is important for the program to know when to stop
c = open(file_path_BMRB)
lines1 = c.readlines()
total_lines = len(lines1)
c.close()

# Read the data
f = open(file_path_BMRB)

#Creates dictionary for this BMRB data
BMRB = {}

#creates handy counters
i = 0
n = 1

#creates handy variables
titles = []
added_amino_acids = []
atoms_list = []

#Starts the loop trhough the file. Line by line.
while i <= total_lines:
    this_line = f.readline()
    i = i + 1
    
    #resets helpful title counter n
    n = 1

    #print(this_line)
    
    #Handligng exceptional lines
    if this_line.startswith('\n') or this_line == '':
        pass
    
    else:    
        #Getting titles to use as keys on dictionary
        if this_line[0].islower():
            #splitting title line
            this_line_split = this_line.split(',')

            #adding items from split line to list titles for further use
            for item in this_line_split:
                #also deletes \n from the last item
                titles.append(item.replace('\n',''))

        #Moving on with reading the BRMB file/adding the data to a dictioanry
        #splitting title line
        this_line_split = this_line.split(',')
        #print(this_line_split)
        
        #After titles where taken care of
        if this_line[0].islower() == 0:
            #checking if entry for current amino acid was already created in the dictionary with the aid of a list
            if this_line_split[0] not in added_amino_acids:
                added_amino_acids.append(this_line_split[0])
                #if entry was not created yet, creates an entry
                BMRB[this_line_split[0]] = {}
                #print('new')                
                #creates sub-entries
                while n < len(titles):
                    #print(titles[n])
                    temporary_BMRB = {}
                    temporary_BMRB[titles[n]] = []
                    #print(temporary_BMRB)
                    BMRB[this_line_split[0]].update(temporary_BMRB)
                    #Moves counter to next iteration of loop
                    n = n + 1
                    
            #resets helpful title counter n
            #print(BMRB)
            n = 1

            #when entry and sub-entries were already created, proceeds to add data
            while n < len(titles):
                BMRB[this_line_split[0]][titles[n]].append(this_line_split[n].replace('\n',''))
                #print(n)
                
                #Adds atom name to atoms_list
                #checks if current iteration of the n-loop is about atoms
                if titles[n] == "atom_id":
                    #Checks if atom was already added to the atoms_list, if not, it will be added
                    if this_line_split[n] not in atoms_list:
                        atoms_list.append(this_line_split[n])
                    #print(this_line_split[n])
                
                #Moves counter to next iteration of loop
                n = n + 1
            
#print(BMRB)
#print(titles)
#print(atoms_list)
#print(added_amino_acids)
print("Done!"+'\n')
    
# ========================================================================== #

#This part of the program asks what is the atom and what is the chemical shift value.
#It will use this information to find the most likely macthes for amino acids

#User selects how many atoms to work with simultaneously:
number_atoms = input("How many atoms are you working with"+"\n")

#Creates master dictionary to stores results for every atom
total_results = {}

#Creates list to keep track of provided atoms
users_atoms_list = []

#Creates handful counter
number = 0

while number < int(number_atoms):
    
    #moves counter to next iteration
    number = number + 1

    #The suer selects atom and respective chemical shift
    atom = input("What is the atom?\nChoose from one of the following:\n"+str(atoms_list)+"\n")
    #makes sure user selec ts a valid atom
    while atom not in atoms_list:
        atom = input("Please choose one of the following atoms:\n"+str(atoms_list)+"\n")
    #asks for chemical shift    
    chemical_shift = input("What is the atom's chemical shift?\n")

    #Creates/resets variables to store results
    results = {'amino_acids':[], 'delta_avg':[], 'within_sd':[], 'delta_avg_squared':[], 'sorted_delta_avg_squared':[]}


    #Calculates most likely amino acids using average and standard deviation
    #Starts looping through amino acids
    for this_amino_acid in added_amino_acids:

        #Checks if select atom exist in current amino acid
        if atom in BMRB[this_amino_acid]['atom_id']:
            #Subtracts provided atom's average from provided chemical shift
            this_delta_avg = float(chemical_shift) - float(BMRB[this_amino_acid]['avg'][BMRB[this_amino_acid]['atom_id'].index(atom)])
            #Finds delta_avg_squared
            this_delta_avg_squared = this_delta_avg**2        
            #Checks if result falls within standard deviation
            if this_delta_avg**2 < float(BMRB[this_amino_acid]['std'][BMRB[this_amino_acid]['atom_id'].index(atom)])**2:
                this_within_sd = "yes"
            else:
                this_within_sd = "no"
            #Stores results
            results['amino_acids'].append(this_amino_acid)
            results['delta_avg'].append(this_delta_avg)
            results['delta_avg_squared'].append(this_delta_avg_squared)
            results['within_sd'].append(this_within_sd)


    #print(results)
    #print(results['amino_acids'])
    #print(results['delta_avg'])
    #print(results['within_sd'])

    # ========================================================================== #

    #This part of the program sorts the results and adds current atom's results to the master dictionary

    #Sorts delta_avg in ascending order and adds it to a list
    results['sorted_delta_avg_squared'] = sorted(results['delta_avg_squared'])
    
    #Adds current results to master dictionary
    total_results[atom] = {}
    total_results[atom].update(results)
    
    #Adds atoms to users_atoms_list
    users_atoms_list.append(atom)

# ========================================================================== #

#This part of the program displays the results to the user

#Starts to loop through the atoms provided by user
for this_atom in users_atoms_list:
    
    #Prints Header
    print("\n\n# ==== # "+str(this_atom)+" # ==== #")
    print("\n"+"AA   Delta_AVG_Squared    Within_SD?")

    #Uses sorted list to ain in printing out results
    for item in total_results[this_atom]['sorted_delta_avg_squared']:
        print("\n"+str(total_results[this_atom]['amino_acids'][total_results[this_atom]['delta_avg_squared'].index(item)])+": "+str(item)+": "+str(total_results[this_atom]['within_sd'][total_results[this_atom]['delta_avg_squared'].index(item)]))


Reading BMRB Data Sheet...

Done!

How many atoms are you working with
2
What is the atom?
Choose from one of the following:
['H', 'HA', 'MB', 'C', 'CA', 'CB', 'N', 'HB2', 'HB3', 'HD2', 'HD3', 'HE', 'HG2', 'HG3', 'HH11', 'HH12', 'HH21', 'HH22', 'CD', 'CG', 'CZ', 'NE', 'NH1', 'NH2', 'HD21', 'HD22', 'ND2', 'HG', 'HE21', 'HE22', 'NE2', 'HE2', 'H1', 'HA2', 'HA3', 'HD1', 'HE1', 'CD2', 'CE1', 'ND1', 'HB', 'HG12', 'HG13', 'MD', 'MG', 'CD1', 'CG1', 'CG2', 'MD1', 'MD2', 'HE3', 'CE', 'NZ', 'QZ', 'ME', 'HZ', 'CE2', 'H2', 'HG1', 'HH2', 'HZ2', 'HZ3', 'CE3', 'CH2', 'CZ2', 'CZ3', 'NE1', 'HH', 'MG1', 'MG2']
H
What is the atom's chemical shift?
8.3
What is the atom?
Choose from one of the following:
['H', 'HA', 'MB', 'C', 'CA', 'CB', 'N', 'HB2', 'HB3', 'HD2', 'HD3', 'HE', 'HG2', 'HG3', 'HH11', 'HH12', 'HH21', 'HH22', 'CD', 'CG', 'CZ', 'NE', 'NH1', 'NH2', 'HD21', 'HD22', 'ND2', 'HG', 'HE21', 'HE22', 'NE2', 'HE2', 'H1', 'HA2', 'HA3', 'HD1', 'HE1', 'CD2', 'CE1', 'ND1', 'HB', 'HG12', 'HG13', 'MD', 'MG', 'C

In [ ]:
#print(BMRB.keys())
#print(added_amino_acids)
#print(BMRB)
print(total_results)
print(users_atoms_list)